In [2]:
import json
import requests
import uuid
import csv
import pandas as pd

In [3]:
artist_rows = []
artwork_rows = []

response = requests.get('https://harvardartmuseums.org/browse?q=&load_amount=100&offset=0')
data = json.loads(response.content)
for record in data['records']:
    try:
        artist_uuid = uuid.uuid4()
        person_id = record['people'][0]['personid']
        artist_url = f'https://harvardartmuseums.org/collections/person/{person_id}'
       
        artist_name = record['people'][0]['name']
        birthplace = record['people'][0]['birthplace']
        gender = record['people'][0]['gender']
      
        artist_row = [artist_uuid, person_id, artist_name, birthplace, gender]
        artist_rows.append(artist_row)
        
        response_object = requests.get(f"https://harvardartmuseums.org/browse?person={person_id}&load_amount=100&offset=0")
        object_data = json.loads(response_object.content)
        for object_record in object_data['records']:
            try:
                object_id = object_record['objectid']
                artwork_uuid = uuid.uuid4()
                object_url = f"https://harvardartmuseums.org/collections/object/{object_id}?position=0&context=person&id={person_id}"
                   
                object_name = object_record['title']
                object_image_url = object_record['images'][0]['baseimageurl']
                classification = object_record['classification']
                technique = object_record['technique']
                description = object_record['description']
                culture = object_record['culture']
               
                artwork_row = [artwork_uuid, object_id, artist_uuid, object_name, object_url, object_image_url, classification, culture, technique, description]
                artwork_rows.append(artwork_row)
         
            except:
                pass
        
        artists_df = pd.DataFrame(artist_rows, columns=['artist_uuid', 'person_id', 'artist_name', 'birthplace', 'gender'])
        artworks_df = pd.DataFrame(artwork_rows, columns=['artwork_uuid', 'object_id', 'artist_uuid', 'object_name', 'object_url', 'object_image_url', 'classification', 'culture', 'technique', 'description'])


        artists_df.to_csv('artists.csv')
        artworks_df.to_csv('artworks.csv')    
        
    except:
        pass